In [105]:
from icecream import ic

In [106]:
# from sktime.networks.base import BaseDeepNetwork


# class MLPNetwork(BaseDeepNetwork):

#     def __init__(
#         self,
#         random_state=0,
#     ):
#         self.random_state = random_state
#         super(MLPNetwork, self).__init__()

#     def build_network(self, input_shape, **kwargs):
#         from tensorflow import keras

#         # flattened because multivariate should be on same axis
#         input_layer = keras.layers.Input(input_shape)
#         input_layer_flattened = keras.layers.Flatten()(input_layer)
#         layer_1 = keras.layers.Dense(100, activation="relu")(input_layer_flattened)

#         return input_layer, layer_1

In [107]:
from sktime.networks.mlp import MLPNetwork
from sktime.utils.validation._dependencies import _check_dl_dependencies

_check_dl_dependencies(severity="warning")


class FCNForecaster:
    """Temp docstring."""

    def __init__(
        self,
        n_epochs=2000,
        batch_size=1,
        steps=3,
        callbacks=None,
        verbose=False,
        loss="mse",
        metrics=None,
        random_state=None,
        activation="relu",
        use_bias=True,
        optimizer=None,
    ):
        _check_dl_dependencies(severity="error")
        super(FCNForecaster, self).__init__()
        self.callbacks = callbacks
        self.n_epochs = n_epochs
        self.batch_size = batch_size
        self.steps = steps
        self.verbose = verbose
        self.loss = loss
        self.metrics = metrics
        self.random_state = random_state
        self.activation = activation
        self.use_bias = use_bias
        self.optimizer = optimizer
        self.history = None
        self._network = MLPNetwork()

    def build_model(self, input_shape, **kwargs):
        """Temp docstring."""
        from tensorflow import keras

        self.metrics = ["accuracy"] if self.metrics is None else self.metrics
        input_layer, output_layer = self._network.build_network(
            input_shape,
        )
        output_layer = keras.layers.Dense(units=1)(output_layer)

        self.optimizer_ = (
            keras.optimizers.Adam(learning_rate=0.0001)
            if self.optimizer is None
            else self.optimizer
        )

        model = keras.models.Model(inputs=input_layer, outputs=output_layer)
        model.compile(loss=self.loss, optimizer=self.optimizer_, metrics=self.metrics)
        return model

    def _predict(self, fh, X=None):
        """Temp docstring."""
        import numpy as np

        currentPred = 1
        lastPred = max(fh)
        fvalues = []
        fh = set(fh)
        source = self.source[-1]
        source = source[np.newaxis, :, :]
        while currentPred <= lastPred:
            ic(source)
            yhat = self.model_.predict(source)
            ic(yhat)
            source = np.delete(source, axis=2, obj=0)
            source = np.insert(source, obj=source.shape[-1], values=yhat, axis=-1)
            if currentPred in fh:
                fvalues.append(yhat)

            currentPred += 1
        return fvalues

    def _fit(self, y, fh=None, X=None):
        """Temp docstring."""
        import numpy as np

        source, target = self.splitSeq(self.steps, y)
        if X is not None:
            src_x, _ = self.splitSeq(self.steps, X)
            # currently takes care of cases where exog data is
            # greater than 1 in length
            source = [
                [_sx + [_sy] for _sx, _sy in zip(sx, sy)]
                for sx, sy in zip(src_x, source)
            ]

        source, target = np.array(source), np.array(target)
        if X is None:
            source = source.reshape((*source.shape, 1))
        source = source.transpose(0, 2, 1)
        self.input_shape = source.shape[1:]
        self.source, self.target = source, target

        self.model_ = self.build_model(self.input_shape)
        if self.verbose:
            self.model_.summary()

        self.history = self.model_.fit(
            source,
            target,
            batch_size=self.batch_size,
            epochs=self.n_epochs,
            verbose=self.verbose,
            callbacks=self.callbacks,
        )
        return self

    def splitSeq(self, steps, seq):
        """Temp."""
        source, target = [], []
        for i in range(len(seq)):
            end_idx = i + steps
            if end_idx > len(seq) - 1:
                break
            seq_src, seq_tgt = seq[i:end_idx], seq[end_idx]
            source.append(seq_src)
            target.append(seq_tgt)
        return source, target

In [108]:
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]

In [109]:
fcn = FCNForecaster()

In [110]:
fcn._fit(raw_seq)

In [111]:
fcn.history.history

{'loss': [4357.99560546875,
  3730.872802734375,
  3082.394775390625,
  3000.588623046875,
  1719.9803466796875,
  1581.5289306640625,
  994.027587890625,
  669.9326171875,
  448.639892578125,
  332.6037902832031,
  312.4143981933594,
  183.7276611328125,
  164.01600646972656,
  267.8992614746094,
  177.99571228027344,
  72.49307250976562,
  325.3884582519531,
  603.9581909179688,
  97.92171478271484,
  270.5317687988281,
  115.1213607788086,
  133.9286346435547,
  69.43212127685547,
  174.29981994628906,
  200.71043395996094,
  59.058231353759766,
  148.9652557373047,
  130.30503845214844,
  127.077880859375,
  254.2949676513672,
  120.7545166015625,
  42.40974807739258,
  118.46422576904297,
  131.0071563720703,
  135.3533172607422,
  556.5653686523438,
  30.840499877929688,
  74.74600982666016,
  191.09654235839844,
  154.0568389892578,
  29.976762771606445,
  77.98247528076172,
  31.43400001525879,
  143.9558868408203,
  67.12779998779297,
  80.41931915283203,
  116.65271759033203,

In [112]:
fcn._predict([1, 2, 3, 4])

ic| source: array([[[60, 70, 80]]])


1/1 [==============================] - 0s 147ms/step


ic| yhat: array([[73.68962]], dtype=float32)
ic| source: array([[[70, 80, 73]]])


1/1 [==============================] - 0s 31ms/step


ic| yhat: array([[70.84698]], dtype=float32)
ic| source: array([[[80, 73, 70]]])


1/1 [==============================] - 0s 31ms/step


ic| yhat: array([[68.76868]], dtype=float32)
ic| source: array([[[73, 70, 68]]])


1/1 [==============================] - 0s 27ms/step


ic| yhat: array([[65.72005]], dtype=float32)


[array([[73.68962]], dtype=float32),
 array([[70.84698]], dtype=float32),
 array([[68.76868]], dtype=float32),
 array([[65.72005]], dtype=float32)]